In [13]:
import torch
#from Ipython import display
from d2l import torch as d2l

batch_size = 256   #每次读取256张图片
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)   #训练集和测试集


In [14]:
#个样本都是的图像。 展平每个图像，把它们看作长度为784的向量
#数据集有10个类别，所以网络输出维度为10。 因此，权重将构成一个784X10的矩阵， 偏置将构成一个1X10的行向量
num_inputs = 784
num_outputs = 10

w = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)   #随机数
b = torch.zeros(num_outputs,requires_grad=True)


In [15]:
#给定一个矩阵，对所有元素求和
X = torch.tensor([[1.0,2.0,3.0],[4.0,5.0,6.0]])
X.sum(0,keepdim=True),X.sum(1,keepdim=True)   #这里sum（0）是去掉行，就是每一列相加，sum（1）
#是去掉列，就是每一行相加

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [16]:
#实现Softmax，这里的x是矩阵，所以对x的每一行做sofemax
def softmax(X):
    X_exp = torch.exp(X)  #对每一个元素做指数运算
    #这里的X参数代入的就是O，O=wx+b
    partition = X_exp.sum(1,keepdim=True) #对每一行的幂求和，按维度为1
    return X_exp / partition     #这里应用了广播机制，对每一行相对的去除以，不够的自动补齐，生成一个向量



In [53]:
#验证
X = torch.normal(0,1,(2,5))   #随机的均值为0，方差为1的2行5列的矩阵
X

tensor([[-1.2206,  0.7038,  0.2594, -0.6951, -3.4736],
        [-0.8395, -1.3028, -1.2051, -1.0631,  0.3856]])

In [54]:
X_prob = softmax(X)   #这里X是一个2X5矩阵，代入后生成的是
X_prob

tensor([[0.0712, 0.4879, 0.3129, 0.1205, 0.0075],
        [0.1532, 0.0964, 0.1063, 0.1225, 0.5216]])

In [55]:
X_prob.sum(1)

tensor([1., 1.])

In [18]:
#实现softmax回归模型
def net(X):
    return softmax(torch.matmul(X.reshape((-1,w.shape[0])),w) + b)
    #使用reshape函数将每张原始图像展平为向量,,,,,-1是指由torch自己去计算
    #这里的X是后面要输入的数据
    # X本身是3维（，，，）向量，这里改成2维，shape【0】是指w的行数，shape[1]是指列数
    #这里的w的行数是784
    #x变成一个256X784的矩阵
    #再对x和w进行矩阵乘法，再加上b   有广播机制
    #获得一个256X10的矩阵
    #再softmax，拿到所有元素值大于0，行为1 的输出。


In [20]:
#定义交叉熵损失函数
#测试：拿出预测值
y = torch.tensor([0,2])   #测试：假设的真实值，，y代表的是样本，总共有2个样本，采用独热编码，非0的那一个元素为真实值
y_hat = torch.tensor([[0.1,0.3,0.6],[0.3,0.2,0.5]])   #测试：假设的预测值
y_hat[[0,1],[0,2]]   #这里的[0,2]就是y，意思是拿出y_hat中第0行0列和第1行2列的元素

tensor([0.1000, 0.5000])

In [29]:
len(y_hat)
range(2)

range(0, 2)

In [32]:
torch.log(y_hat[[0,1],[0,2]])

tensor([-2.3026, -0.6931])

In [26]:
#实现交叉熵损失函数
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])
    
    #y_hat是预测值，y是真实值
    #首先取len（y_hat)，y_hat的长度，就是取0到 n的向量
    #然后取对应真实标号的预测值，

cross_entropy(y_hat,y)  #y_hat是2X3的预测值，y是长为2的向量


tensor([2.3026, 0.6931])

#将预测类别与真实y元素进行比较
def accuracy(y_hat,y):
    #计算预测正确的数量
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:   #如果y_hat的维度是2，列数大于1的时候
        y_hat = y_hat.argmax(axis = 1)   #找每一行元素最大的那个下标
    cmp = y_hat.type(y.dtype) == y     #y_hat转成y的数据类型，cmp是布尔类型的向量
    return float(cmp.type(y.dtype).sum())  #cmp是布尔向量，所以要求和

accuracy(y_hat,y) / len(y)


In [47]:
class Accumulator:  #@save
    """在n个变量上累加"""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [51]:
#评估任意模型的准确率---就是计算模型在数据迭代器上的精度

def evaluate_accuracy(net,data_iter):
    #计算在制定数据集上模型的精度
    if isinstance(net,torch.nn.Module):
        net.eval()   #将模型设置为评估模式
    metric = Accumulator(2)   #正确预测数、预测总数
    for X,y in data_iter:   #data_iter-----迭代器，从迭代器中一次取出X,y
        metric.add(accuracy(net(X),y),y.numel())   #accuracy(net(X),y)--预测正确的样本数，，y.numel()样本总数
    return metric[0] / metric[1]   #分类正确的样本数/总样本数 = 精度

In [52]:

evaluate_accuracy(net, test_iter) #这里的test_iter--测试迭代器是torch自带的

0.1175

In [ ]:
#softmax回归的训练
def train_epoch_ch3(net,train_iter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.train()
    metric = Accumulator(3)
    for X,y in train_iter:
        y_hat = net(X)
        l = loss(y_hat,y)
        if isinstance(updater,torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            updater.step()
            metric.add(
                float(l)*len(y),accuracy(y_hat,y),
                y.size().numel()
            )
        else:
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()),accuracy(y_hat,y))
    return metric[0] / metric[]